In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import shutil
import glob
import PIL
from keras.models import load_model

In [2]:
weights_batsman_detection = "./YOLOv5/batsman_detection_kaggle.pt"
weights_ball_detection = "./YOLOV5/ball_detection_kaggle.pt"

In [3]:
model_l_r = load_model("./Left or right")
model_batsman_detection = torch.hub.load("./YOLOv5/yolov5/", source="local",model="custom", path=weights_batsman_detection)
model_ball_detection = torch.hub.load("./YOLOv5/yolov5/", source="local",model="custom", path=weights_ball_detection)

YOLOv5  2022-10-9 Python-3.8.8 torch-1.11.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5  2022-10-9 Python-3.8.8 torch-1.11.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
dataset = "../Dataset/"
temp_images = "./Integration (Ball Tracking + Left Right)/Temp Images/"
org_frames = "./Integration (Ball Tracking + Left Right)/Org Frames/"

In [5]:
def detect_left_right(img_path): # Image will be grayscale
    img = cv2.imread(img_path)
    img = cv2.resize(img, (124, 89))
    
    data = {"Input": [img]}
    df = pd.DataFrame(data)
    test = df["Input"].values
    test_list = [test[0]]
    
    test = np.array(test_list)
    img = tf.convert_to_tensor(np.array(test))
    
    predictions = model_l_r.predict([img])
    left = predictions[0][0]
    right = predictions[0][1]
        
    if left > right:
        return "left"
    elif right > left:
        return "right"
    else:
        return "unsure"

In [10]:
def inference(vid_path):
    try:
        os.mkdir("./Integration (Ball Tracking + Left Right)/Org Frames")
        os.mkdir("./Integration (Ball Tracking + Left Right)/Temp Images")
        os.mkdir("./Integration (Ball Tracking + Left Right)/Cropped Images")
    except:
        pass

    vidCapture = cv2.VideoCapture(vid_path)

    count_frames = 0
    left_right_lst = []
    res_lst = []
    
    while(True):
        ret, frame = vidCapture.read()

        if ret is False:
            break
        
        cv2.imwrite(org_frames + str(count_frames) + ".jpg", frame)
        
        if (count_frames<15):

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(temp_images + str(count_frames) + ".jpg", frame)

            prediction_batsman = model_batsman_detection(temp_images + str(count_frames) + ".jpg")

            try:
                prediction_coordinates = prediction_batsman.pandas().xyxy[0].sort_values(by="confidence", ascending=False).iloc[0, :]
            except Exception as e:
                continue

            x_min = int(prediction_coordinates["xmin"])
            y_min = int(prediction_coordinates["ymin"])
            x_max = int(prediction_coordinates["xmax"])
            y_max = int(prediction_coordinates["ymax"])

            cropped_img = []
            for x_index in range(y_min, y_max):
                cropped_img.append(frame[x_index])

            cropped_img_final = []
            for x_crop in cropped_img:
                new_crop = []
                for y_index in range(x_min, x_max):
                    new_crop.append(x_crop[y_index])
                cropped_img_final.append(np.array(new_crop))


            cv2.imwrite("./Integration (Ball Tracking + Left Right)/Cropped Images/"+ str(count_frames) + "_cropped.jpg", np.array(cropped_img_final))


            res = detect_left_right("./Integration (Ball Tracking + Left Right)/Cropped Images/"+ str(count_frames) + "_cropped.jpg")
            left_right_lst.append(res)

        count_frames += 1
    
    frame_lst_len = len(glob.glob(org_frames + "*.jpg"))
    sorted_lst = []
    for i in range(frame_lst_len):
        sorted_lst.append(str(i) + ".jpg")
        
    
    for i in sorted_lst:
        path = "./Integration (Ball Tracking + Left Right)/Org Frames/" + i

        pred = model_ball_detection(path)
        try:
            best_ball = pred.pandas().xyxy[0].sort_values(by="confidence", ascending=False).iloc[0, :]
            if (best_ball["confidence"] > 0.3):
                temp_dict = {"xmin": best_ball["xmin"],
                             "ymin": best_ball["ymin"],
                             "xmax": best_ball["xmax"],
                             "ymax": best_ball["ymax"],
                              "conf": best_ball["confidence"]}
                res_lst.append(temp_dict)
        except:
            pass
        
    x_lst = []
    y_lst = []

    penalty = 0
    for elem in res_lst:
        x_lst.append(elem["xmax"])
        y_lst.append(elem["ymax"] + penalty)
        penalty += 100
    
    x1 = x_lst[0]
    y1 = y_lst[0]
    x2 = x_lst[-1]
    y2 = y_lst[-1]

    slope = ((y2-y1) / (x2-x1))
    left_count = left_right_lst.count("left")
    right_count = left_right_lst.count("right")
    
    shot_style = "Unsure"
    
    shutil.rmtree("./Integration (Ball Tracking + Left Right)/Temp Images")
    shutil.rmtree("./Integration (Ball Tracking + Left Right)/Org Frames")
    shutil.rmtree("./Integration (Ball Tracking + Left Right)/Cropped Images")

    if right_count > left_count:
        if slope < 0:
            shot_style = "Off side"
        else:
            shot_style = "Leg side"
            
        return ["Right handed", shot_style]
    elif left_count > right_count:
        if slope > 0:
            shot_style = "Off side"
        else:
            shot_style = "Leg side"
            
        return ["Left Handed", shot_style]
    else:
        return ["Unsure", shot_style]

In [ ]:
"C:\Users\Assus\Cricket NLP\Cricket NLP Version 1.1\Dataset\8qXVwlm9eGQ\53_59_Trim.mp4"

In [12]:
result = inference('../Dataset/8qXVwlm9eGQ/53_59_Trim.mp4')
result

['Right handed', 'Off side']